<a href="https://colab.research.google.com/github/CRAUGUTH/aiProject/blob/main/AI_Project_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Training NN**


In [ ]:
# Task 1: Install Libraries
!pip install --upgrade spotipy torch pandas scikit-learn transformers librosa

# Task 2: Import Libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
import joblib

In [ ]:
# Task 3: Define Your Model with more layers and dropout
class SongMoodClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SongMoodClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.network(x)

In [ ]:
# Task 4: Load and Preprocess Data
df = pd.read_csv('/content/music_moods.csv')
features = df.drop(['id', 'mood'], axis=1).values
labels = df['mood'].values

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

X_train, X_val, y_train, y_val = train_test_split(features_scaled, labels_encoded, test_size=0.2, random_state=42)

In [ ]:
# Task 5: Train and Validate the Model
input_size = X_train.shape[1]
num_classes = len(np.unique(labels_encoded))

model = SongMoodClassifier(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 65  # You might need to adjust the batch size depending on your dataset

# Function to create batches
def create_batches(X, y, batch_size):
    for i in range(0, len(X), batch_size):
        yield X[i:i+batch_size], y[i:i+batch_size]

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in create_batches(X_train, y_train, batch_size):
        inputs = torch.FloatTensor(X_batch)
        targets = torch.LongTensor(y_batch)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Validation loop
    if epoch % 10 == 0:  # Validate every 10 epochs
        model.eval()
        val_losses = []
        val_accuracies = []
        for X_batch, y_batch in create_batches(X_val, y_val, batch_size):
            inputs = torch.FloatTensor(X_batch)
            targets = torch.LongTensor(y_batch)
            with torch.no_grad():
                outputs = model(inputs)
                val_loss = criterion(outputs, targets)
                val_losses.append(val_loss.item())

                _, predicted = torch.max(outputs, 1)
                val_accuracy = (predicted == targets).sum().item() / targets.size(0)
                val_accuracies.append(val_accuracy)

        avg_val_loss = np.mean(val_losses)
        avg_val_accuracy = np.mean(val_accuracies)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss}, Validation Accuracy: {avg_val_accuracy}')

In [ ]:
# Task 6: Save the Model and Preprocessors
torch.save(model.state_dict(), 'song_mood_classifier.pth')
joblib.dump(label_encoder, 'label_encoder.joblib')
joblib.dump(scaler, 'scaler.joblib')

# **Music Classification**

# **Playlist Generation**